In [1]:
import requests
import os
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from dotenv import load_dotenv



### No Tor

In [2]:
import requests

proxies = {
    "http": "socks5h://127.0.0.1:9050",
    "https": "socks5h://127.0.0.1:9050"
}

response = requests.get("https://check.torproject.org", proxies=proxies)
print(response.text)

<!doctype html>
<html lang="en_US">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />
  <title>
    
      Congratulations. This browser is configured to use Tor.
    
</title>
  <link rel="icon" type="image/x-icon" href="/torcheck/img/tor-not.png" />
  <style>
    html { height: 100%; }
    body {
      height: 100%;
      text-align: center;
      font-family: Helvetica, sans-serif;
      margin: 0;
    }
    h1 { font-size: 2em; }
    a { text-decoration: none; }
    a:hover { text-decoration: underline; }
    ul { margin: 0; padding: 0; }
    li {
      text-align: left;
      display: inline;
      list-style-type: none;
      margin: 0;
    }
    .content {
      padding: 4em;
    }
    .foot {
      padding: 2em 4em;
      clear: both;
    }
    .head {
      background-color: #85559f;
      padding: 0.8em 0;
      color: #fff;
    }
    .project {
      margin: 0;
      font-size: 0.8em;
    }
    
  .on { color: green;

In [3]:
# Z-library search url 
search_url = "https://en.z-lib.gs/s/?content_type=book&languages%5B0%5D=afrikaans&languages%5B1%5D=amharic&languages%5B2%5D=hausa&languages%5B3%5D=igbo&languages%5B4%5D=kinyarwanda&languages%5B5%5D=malagasy&languages%5B6%5D=oromo&languages%5B7%5D=shona&languages%5B8%5D=somali&languages%5B9%5D=southern_sotho&languages%5B10%5D=swahili&languages%5B11%5D=tigrinya&languages%5B12%5D=xhosa&languages%5B13%5D=yoruba&languages%5B14%5D=zulu&page=1"
headers = {"User-Agent": "Mozilla/5.0"}

In [3]:
response = requests.get(search_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
books = soup.find_all("div", class_="book-item")
books[:2]

In [6]:
books = soup.find_all("div", class_="book-item")
book_links = []
langauges = []
titles = []


for book in books:
    book_card = book.find("z-bookcard")  # Find the <z-bookcard> tag
    if book_card:
        book_href = book_card.get("href")  # Extract the 'href' attribute
        if book_href:
            full_url = f"https://en.z-lib.gs{book_href}"  # Construct full URL
            book_links.append(full_url)
        language = book_card.get("language")
        langauges.append(language) if language else langauges.append("Unknown")
        title = book_card.find("div", {"slot": "title"}).text
        titles.append(title) if title else titles.append("Unknown")

In [7]:
titles[:10]

['የመጽሐፍ ቅዱስ መዝገበ ቃላት - A Dictionary of the Holy Bible',
 'Church Growth Principles',
 '150 stories (Afrikaans Edition)',
 'የኢትዮጵያ ታሪክ : ኑብያ-አክሱም ዛጕዬ እስከ ዐፄ ይኵኖ አምላክ ዘመነ መንግሥት',
 'ዐፄ ምኒልክ እና የኢትዮጵያ አንድነት',
 'ዐፄ ዮሐንስ እና የኢትዮጵያ አንድነት',
 'Die vrou in die blou mantel (Afrikaans Edition)',
 'ታሪክና ምሳሌ ፩ኛ መጽሐፍ - Stories and Parables, 1st book',
 'ታሪክና ምሳሌ ፪ኛ መጽሐፍ - Stories and Parables, 2nd book',
 'Spaghetti tussen jou tone (Afrikaans Edition)']

In [11]:
len(book_titles)

50

### Books

In [ ]:
BASE_URL ="https://en.z-lib.gs/s/?content_type=book&languages%5B0%5D=afrikaans&languages%5B1%5D=amharic&languages%5B2%5D=hausa&languages%5B3%5D=igbo&languages%5B4%5D=kinyarwanda&languages%5B5%5D=malagasy&languages%5B6%5D=oromo&languages%5B7%5D=shona&languages%5B8%5D=somali&languages%5B9%5D=southern_sotho&languages%5B10%5D=swahili&languages%5B11%5D=tigrinya&languages%5B12%5D=xhosa&languages%5B13%5D=yoruba&languages%5B14%5D=zulu&page={page}"
headers = {"User-Agent": "Mozilla/5.0"}

# List to store book data 
all_books = []

# max pages to scrape
MAX_PAGES = 20

for page in range(1, MAX_PAGES+1):
    print (f"Scraping page {page}...")
    url = BASE_URL.format(page=page)
    response = requests.get(url, headers=headers)

    # # Check if the request was successful
    # if response.status_code != 200:
    #     print(f"⚠️ Failed to fetch page {page}. Status Code: {response.status_code}")
    #     break

    soup = BeautifulSoup(response.text, 'html.parser')


    
    books = soup.find_all("div", class_="book-item")

    if not books:
        print("No more books found.")
        break

    for book in books:
        title_tag = book.find('div', {'slot':'title'}).text
        book_card = book.find("z-bookcard")
        link_tag = book_card.get("href") if book_card else None
        language_tag = book_card.get("language") if book_card else None
        download_tag = book_card.get("download") if book_card else None
        
        
        if title_tag:
            title = title_tag.strip()
            language = language_tag.strip() if language_tag else "Unknown"
            link = f"https://en.z-lib.gs{link_tag.strip()}" if link_tag else "Unknown"
            download = f"https://en.z-lib.gs{download_tag.strip()}" if download_tag else "Unknown"
            all_books.append({"Title": title, "Language": language, "Link": link, "Download": download})

    time.sleep(2)

csv_filename = "african_language_books.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["Title", "Language", "Link", "Download"])
    writer.writeheader()
    writer.writerows(all_books)

# Print summary
print(f"\n✅ Scraping completed! Total Books Found: {len(all_books)}")
print(f"📄 Data saved in '{csv_filename}'")

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
No more books found.

✅ Scraping completed! Total Books Found: 150
📄 Data saved in 'african_language_books.csv'


In [8]:
df = pd.read_csv("african_language_books.csv")

In [9]:
df.head(20)

,Title,Language,Link,Download
0,የመጽሐፍ ቅዱስ መዝገበ ቃላት - A Dictionary of the Holy ...,amharic,https://en.z-lib.gs/book/1053818/53c4d5/%E1%8B...,https://en.z-lib.gs/dl/1053818/251e31
1,Church Growth Principles,swahili,https://en.z-lib.gs/book/16740533/fd6164/churc...,https://en.z-lib.gs/dl/16740533/61136a
2,150 stories (Afrikaans Edition),afrikaans,https://en.z-lib.gs/book/5351114/c575d9/150-st...,https://en.z-lib.gs/dl/5351114/0531a9
3,የኢትዮጵያ ታሪክ : ኑብያ-አክሱም ዛጕዬ እስከ ዐፄ ይኵኖ አምላክ ዘመነ ...,amharic,https://en.z-lib.gs/book/1217944/84e012/%E1%8B...,https://en.z-lib.gs/dl/1217944/5a3ef5
4,ዐፄ ምኒልክ እና የኢትዮጵያ አንድነት,amharic,https://en.z-lib.gs/book/1055276/19dcb5/%E1%8B...,https://en.z-lib.gs/dl/1055276/d52504
5,ዐፄ ዮሐንስ እና የኢትዮጵያ አንድነት,amharic,https://en.z-lib.gs/book/1055272/a646e7/%E1%8B...,https://en.z-lib.gs/dl/1055272/a5683c
6,Die vrou in die blou mantel (Afrikaans Edition),afrikaans,https://en.z-lib.gs/book/5683451/fd0dde/die-vr...,https://en.z-lib.gs/dl/5683451/dadd07
7,"ታሪክና ምሳሌ ፩ኛ መጽሐፍ - Stories and Parables, 1st book",amharic,https://en.z-lib.gs/book/1053824/5c8272/%E1%89...,https://en.z-lib.gs/dl/1053824/461da8
8,"ታሪክና ምሳሌ ፪ኛ መጽሐፍ - Stories and Parables, 2nd book",amharic,https://en.z-lib.gs/book/1054377/1be134/%E1%89...,https://en.z-lib.gs/dl/1054377/483c27
9,Spaghetti tussen jou tone (Afrikaans Edition),afrikaans,https://en.z-lib.gs/book/5659534/4afd5b/spaghe...,https://en.z-lib.gs/dl/5659534/bb133c


In [2]:

# Set up Selenium WebDriver (ensure you have chromedriver installed)
options = Options()
options.add_argument("--headless")  # Run Chrome in headless mode
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

service = Service("C:/Program Files/Google/Chrome/Application/chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

# Base URL for book search
BASE_URL = "https://en.z-lib.gs/s/?content_type=book&languages%5B0%5D=afrikaans&languages%5B1%5D=amharic&languages%5B2%5D=hausa&languages%5B3%5D=igbo&languages%5B4%5D=kinyarwanda&languages%5B5%5D=malagasy&languages%5B6%5D=oromo&languages%5B7%5D=shona&languages%5B8%5D=somali&languages%5B9%5D=southern_sotho&languages%5B10%5D=swahili&languages%5B11%5D=tigrinya&languages%5B12%5D=xhosa&languages%5B13%5D=yoruba&languages%5B14%5D=zulu&page={page}"

# List to store book data
all_books = []

# Max pages to scrape
MAX_PAGES = 5

for page in range(1, MAX_PAGES + 1):
    print(f"🔍 Scraping page {page}...")

    # Open the page with Selenium
    driver.get(BASE_URL.format(page=page))
    time.sleep(5)  # Wait for JavaScript to fully load

    # Get the fully rendered HTML
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Find all books
    books = soup.find_all("div", class_="book-item")

    if not books:
        print("✅ No more books found.")
        break

    for book in books:
        title_tag = book.find('div', {'slot':'title'}).text
        book_card = book.find("z-bookcard")
        link_tag = book_card.get("href") if book_card else None
        language_tag = book_card.get("language") if book_card else None
        download_tag = book_card.get("download") if book_card else None
        
        
        if title_tag:
            title = title_tag.strip()
            language = language_tag.strip() if language_tag else "Unknown"
            link = f"https://en.z-lib.gs{link_tag.strip()}" if link_tag else "Unknown"
            download = f"https://en.z-lib.gs{download_tag.strip()}" if download_tag else "Unknown"
            all_books.append({"Title": title, "Language": language, "Link": link, "Download": download})

    time.sleep(2)


# Close Selenium WebDriver
driver.quit()

# Save results to CSV
csv_filename = "african_language_books_selenium.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["Title", "Author", "Language", "Link", "Download"])
    writer.writeheader()
    writer.writerows(all_books)

# Print summary
print(f"\n✅ Scraping completed! Total Books Found: {len(all_books)}")
print(f"📄 Data saved in '{csv_filename}'")


🔍 Scraping page 1...
🔍 Scraping page 2...
🔍 Scraping page 3...
🔍 Scraping page 4...
✅ No more books found.

✅ Scraping completed! Total Books Found: 150
📄 Data saved in 'african_language_books_selenium.csv'


In [3]:
df2 = pd.read_csv("african_language_books_selenium.csv")
df2.head(20)

,Title,Author,Language,Link,Download
0,የመጽሐፍ ቅዱስ መዝገበ ቃላት - A Dictionary of the Holy ...,NaN,amharic,https://en.z-lib.gs/book/1053818/53c4d5/%E1%8B...,https://en.z-lib.gs/dl/1053818/985c07
1,Church Growth Principles,NaN,swahili,https://en.z-lib.gs/book/16740533/fd6164/churc...,https://en.z-lib.gs/dl/16740533/3b9744
2,150 stories (Afrikaans Edition),NaN,afrikaans,https://en.z-lib.gs/book/5351114/c575d9/150-st...,https://en.z-lib.gs/dl/5351114/aa2959
3,የኢትዮጵያ ታሪክ : ኑብያ-አክሱም ዛጕዬ እስከ ዐፄ ይኵኖ አምላክ ዘመነ ...,NaN,amharic,https://en.z-lib.gs/book/1217944/84e012/%E1%8B...,https://en.z-lib.gs/dl/1217944/0850e7
4,ዐፄ ምኒልክ እና የኢትዮጵያ አንድነት,NaN,amharic,https://en.z-lib.gs/book/1055276/19dcb5/%E1%8B...,https://en.z-lib.gs/dl/1055276/cd4230
5,ዐፄ ዮሐንስ እና የኢትዮጵያ አንድነት,NaN,amharic,https://en.z-lib.gs/book/1055272/a646e7/%E1%8B...,https://en.z-lib.gs/dl/1055272/2ff41f
6,Die vrou in die blou mantel (Afrikaans Edition),NaN,afrikaans,https://en.z-lib.gs/book/5683451/fd0dde/die-vr...,https://en.z-lib.gs/dl/5683451/199434
7,"ታሪክና ምሳሌ ፩ኛ መጽሐፍ - Stories and Parables, 1st book",NaN,amharic,https://en.z-lib.gs/book/1053824/5c8272/%E1%89...,https://en.z-lib.gs/dl/1053824/852fa7
8,"ታሪክና ምሳሌ ፪ኛ መጽሐፍ - Stories and Parables, 2nd book",NaN,amharic,https://en.z-lib.gs/book/1054377/1be134/%E1%89...,https://en.z-lib.gs/dl/1054377/cb8567
9,Spaghetti tussen jou tone (Afrikaans Edition),NaN,afrikaans,https://en.z-lib.gs/book/5659534/4afd5b/spaghe...,https://en.z-lib.gs/dl/5659534/11f65e


In [10]:


def wait_for_download(download_folder, timeout=60):
    """
    Waits until a new file appears in the download folder and is fully downloaded.
    
    :param download_folder: The folder where the file is being downloaded.
    :param timeout: Maximum time (in seconds) to wait for the download to complete.
    """
    start_time = time.time()
    initial_files = set(os.listdir(download_folder))  # Get list of files before download starts
    
    while True:
        current_files = set(os.listdir(download_folder))
        new_files = current_files - initial_files  # Find newly added files

        # If a new file appears, check if it's still downloading (.crdownload on Chrome)
        if new_files:
            for file in new_files:
                file_path = os.path.join(download_folder, file)
                if file.endswith(".crdownload"):
                    print(f"⏳ Waiting for {file} to finish downloading...")
                else:
                    print(f"✅ Download complete: {file}")
                    return file_path  # Return the downloaded file path

        # Check if timeout is exceeded
        if time.time() - start_time > timeout:
            print("⚠️ Download timeout exceeded!")
            return None

        time.sleep(1)  # Check every second


In [ ]:
# 🔹 Set Your Z-Library Credentials
ZLIBRARY_EMAIL = os.getenv("ZLIBRARY_EMAIL") 
ZLIBRARY_PASSWORD = os.getenv("ZLIBRARY_PASSWORD")

# Set up Selenium WebDriver (ensure you have chromedriver installed)
# PATH = 'C:/Users/User/Documents/Kosi/projects/zlibrary/zlibrary'  # Folder to save downloaded files
options = Options()
# prefs = {
#     "download.default_directory": PATH,  # Set Google Drive as the default directory
#     "download.prompt_for_download": False,  # Disable pop-up prompt
#     "download.directory_upgrade": True,
#     "safebrowsing.enabled": True
# }
# options.add_argument("--headless")  # Run Chrome in headless mode
# options.add_experimental_option("prefs", prefs)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

service = Service("C:/Program Files/Google/Chrome/Application/chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

# 🔹 Step 1: Open the Z-Library Login Page
login_url = "https://en.z-lib.gs/login?redirectUrl=https%253A%252F%252Fen.z-lib.gs%252F"  # Z-Library's login page
driver.get(login_url)
time.sleep(3)  # Wait for the page to load

# 🔹 Step 2: Enter Email
email_input = driver.find_element(By.NAME, "email")  # Locate email input field
email_input.send_keys(ZLIBRARY_EMAIL)

# 🔹 Step 3: Enter Password
password_input = driver.find_element(By.NAME, "password")  # Locate password input field
password_input.send_keys(ZLIBRARY_PASSWORD)

# 🔹 Step 4: Click the Login Button
login_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
login_button.click()

# Wait for login to complete
time.sleep(5)

# 🔹 Step 5: Verify Login Was Successful
if "profile" in driver.current_url:  # URL changes after login
    print("✅ Successfully logged into Z-Library!")
else:
    print("❌ Login failed! Check your email and password.")


# Base Search URL for African language books
BASE_URL = "https://en.z-lib.gs/s/?content_type=book&languages%5B0%5D=afrikaans&languages%5B1%5D=amharic&languages%5B2%5D=hausa&languages%5B3%5D=igbo&languages%5B4%5D=kinyarwanda&languages%5B5%5D=malagasy&languages%5B6%5D=oromo&languages%5B7%5D=shona&languages%5B8%5D=somali&languages%5B9%5D=southern_sotho&languages%5B10%5D=swahili&languages%5B11%5D=tigrinya&languages%5B12%5D=xhosa&languages%5B13%5D=yoruba&languages%5B14%5D=zulu&page={page}"

# Maximum number of pages to scrape
MAX_PAGES = 5

# Scrape Z-Library Pages and Download Books
for page in range(1, MAX_PAGES + 1):
    print(f"🔍 Scraping search results from page {page}...")
    
    # Open the search page in Selenium
    driver.get(BASE_URL.format(page=page))
    time.sleep(5)  # Wait for JavaScript to load
    
    # Get the fully rendered HTML
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Find all book items
    books = soup.find_all("div", class_="book-item")

    if not books:
        print("✅ No more books found.")
        break

    # Iterate through books to get the book page link
    for book in books:
        book_card = book.find("z-bookcard")

        if book_card:
            title = book_card.find("div", {"slot": "title"}).text.strip() if book_card.find("div", {"slot": "title"}) else "Unknown"
            language = book_card.get("language") if book_card.get("language") else "Unknown"
            book_link = f"https://en.z-lib.gs{book_card.get('href')}" if book_card.get("href") else "Unknown"

            # 🔹 Step 5: Visit Book Page to Extract the Download Link
            if book_link != "Unknown":
                driver.get(book_link)
                time.sleep(5)

                # Extract new HTML content after navigation
                book_soup = BeautifulSoup(driver.page_source, "html.parser")

                # Find the actual download link
                download_button = book_soup.find("a", class_="btn btn-default addDownloadedBook")
                if download_button:
                    download_link = f"https://en.z-lib.gs{download_button.get('href')}"

                    # 🔹 Step 6: Initiate the File Download
                    driver.get(download_link)
                    time.sleep(5)  # Wait for the download to start
                    print(f"📥 Downloading: {title}...")

                    # Wait until the download is complete
                    wait_for_download(PATH)




    # Sleep between pages to avoid rate limiting
    time.sleep(2)

# Close the Selenium WebDriver
driver.quit()

print("\n✅ All books have been downloaded into Google Drive!")

### Using Tor 

In [2]:


def wait_for_download(download_folder, timeout=60):
    """
    Waits until a new file appears in the download folder and is fully downloaded.
    
    :param download_folder: The folder where the file is being downloaded.
    :param timeout: Maximum time (in seconds) to wait for the download to complete.
    """
    start_time = time.time()
    initial_files = set(os.listdir(download_folder))  # Get list of files before download starts
    
    while True:
        current_files = set(os.listdir(download_folder))
        new_files = current_files - initial_files  # Find newly added files

        # If a new file appears, check if it's still downloading (.crdownload on Chrome)
        if new_files:
            for file in new_files:
                file_path = os.path.join(download_folder, file)
                if file.endswith(".crdownload"):
                    print(f"⏳ Waiting for {file} to finish downloading...")
                else:
                    print(f"✅ Download complete: {file}")
                    return file_path  # Return the downloaded file path

        # Check if timeout is exceeded
        if time.time() - start_time > timeout:
            print("⚠️ Download timeout exceeded!")
            return None

        time.sleep(1)  # Check every second


In [3]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import stem
import stem.control


# ✅ Step 1: Ensure Tor is Running Before Running Selenium
# Open CMD and run:
# "C:\Users\User\Desktop\Tor Browser\Browser\TorBrowser\Tor\tor.exe"

# ✅ Step 2: Configure Tor Proxy for Selenium
TOR_PROXY = "socks5://127.0.0.1:9050"
TOR_CONTROL_PORT = 9051

# # ✅ Step 3: Set Your Z-Library Credentials
# ZLIBRARY_EMAIL = os.getenv("ZLIBRARY_EMAIL") 
# ZLIBRARY_PASSWORD = os.getenv("ZLIBRARY_PASSWORD")

# ✅ Step 3: Function to Request New Tor IP Without Authentication
def change_tor_ip():
    try:
        with stem.control.Controller.from_port(port=TOR_CONTROL_PORT) as controller:
            controller.authenticate()  # No password needed because `CookieAuthentication 0`
            controller.signal(stem.Signal.NEWNYM)  # Request new identity
        time.sleep(5)  # Wait for the IP change to take effect
        print("🔄 Tor IP changed successfully!")
    except Exception as e:
        print(f"❌ Failed to change Tor IP: {e}")

# ✅ Step 4: Set the Download Path
PATH = "C:\\Users\\User\\Downloads"  # Folder to save downloaded files

# # ✅ Step 4: Configure Selenium with Tor Proxy
options = Options()
options.add_argument(f"--proxy-server={TOR_PROXY}")  # Route through Tor
# options.add_argument("--headless")  # Run in headless mode
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# # ✅ Step 5: Initialize Selenium WebDriver
service = Service("C:/Program Files/Google/Chrome/Application/chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

# # ✅ Step 6: Open the Z-Library Login Page
# login_url = "https://en.z-lib.gs/login?redirectUrl=https%253A%252F%252Fen.z-lib.gs%252F"
# driver.get(login_url)
# time.sleep(5)  # Wait for the page to load

# # ✅ Step 7: Enter Email & Password
# try:
#     email_input = driver.find_element(By.NAME, "email")
#     email_input.send_keys(ZLIBRARY_EMAIL)

#     password_input = driver.find_element(By.NAME, "password")
#     password_input.send_keys(ZLIBRARY_PASSWORD)

#     # ✅ Click Login Button
#     login_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
#     login_button.click()
#     time.sleep(5)

#     # ✅ Verify Login Success
#     if "profile" in driver.current_url:
#         print("✅ Successfully logged into Z-Library via Tor!")
#     else:
#         print("❌ Login failed! Check your credentials.")

# except Exception as e:
#     print(f"❌ Error during login: {e}")

# ✅ Step 8: Scrape Z-Library Books
BASE_URL = "https://en.z-lib.gs/s/?content_type=book&languages%5B0%5D=afrikaans&languages%5B1%5D=amharic&languages%5B2%5D=hausa&languages%5B3%5D=igbo&languages%5B4%5D=kinyarwanda&languages%5B5%5D=malagasy&languages%5B6%5D=oromo&languages%5B7%5D=shona&languages%5B8%5D=somali&languages%5B9%5D=southern_sotho&languages%5B10%5D=swahili&languages%5B11%5D=tigrinya&languages%5B12%5D=xhosa&languages%5B13%5D=yoruba&languages%5B14%5D=zulu&page={page}"

MAX_PAGES = 5

for page in range(1, MAX_PAGES + 1):
    print(f"🔍 Scraping search results from page {page} via Tor...")

    driver.get(BASE_URL.format(page=page))
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    books = soup.find_all("div", class_="book-item")

    if not books:
        print("✅ No more books found.")
        break

    for book in books:
        book_card = book.find("z-bookcard")

        if book_card:
            title = book_card.find("div", {"slot": "title"}).text.strip() if book_card.find("div", {"slot": "title"}) else "Unknown"
            language = book_card.get("language") if book_card.get("language") else "Unknown"
            book_link = f"https://en.z-lib.gs{book_card.get('href')}" if book_card.get("href") else "Unknown"

            # ✅ Step 9: Visit Book Page
            if book_link != "Unknown":
                driver.get(book_link)
                time.sleep(5)

                book_soup = BeautifulSoup(driver.page_source, "html.parser")

            # Find the actual download link
                download_button = book_soup.find("a", class_="btn btn-default addDownloadedBook")
                if download_button:
                    download_link = f"https://en.z-lib.gs{download_button.get('href')}"

                    # 🔹 Step 6: Initiate the File Download
                    driver.get(download_link)
                    time.sleep(5)  # Wait for the download to start
                    print(f"📥 Downloading: {title}...")

                    # Wait until the download is complete
                    wait_for_download(PATH)

                    # ✅ Step 7: Change Tor IP After Download
                    change_tor_ip()

    # ✅ Pause to Avoid Rate-Limiting
    time.sleep(3)

# ✅ Close the Selenium WebDriver
driver.quit()
print("\n✅ All books have been downloaded via Tor!")


🔍 Scraping search results from page 1 via Tor...
📥 Downloading: የመጽሐፍ ቅዱስ መዝገበ ቃላት - A Dictionary of the Holy Bible...
⚠️ Download timeout exceeded!
🔄 Tor IP changed successfully!
📥 Downloading: Church Growth Principles...
⚠️ Download timeout exceeded!
🔄 Tor IP changed successfully!


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=133.0.6943.142)
Stacktrace:
	GetHandleVerifier [0x00007FF78D78C6A5+28789]
	(No symbol) [0x00007FF78D6F5B20]
	(No symbol) [0x00007FF78D588F9A]
	(No symbol) [0x00007FF78D574D75]
	(No symbol) [0x00007FF78D599784]
	(No symbol) [0x00007FF78D60F0BF]
	(No symbol) [0x00007FF78D62F0D2]
	(No symbol) [0x00007FF78D607103]
	(No symbol) [0x00007FF78D5CFFC0]
	(No symbol) [0x00007FF78D5D1273]
	GetHandleVerifier [0x00007FF78DAD1AED+3458237]
	GetHandleVerifier [0x00007FF78DAE829C+3550316]
	GetHandleVerifier [0x00007FF78DADDB9D+3507565]
	GetHandleVerifier [0x00007FF78D852C6A+841274]
	(No symbol) [0x00007FF78D7009EF]
	(No symbol) [0x00007FF78D6FCB34]
	(No symbol) [0x00007FF78D6FCCD6]
	(No symbol) [0x00007FF78D6EC119]
	BaseThreadInitThunk [0x00007FFB8CADE8D7+23]
	RtlUserThreadStart [0x00007FFB8D9FBF2C+44]
